https://adresse.data.gouv.fr/

In [5]:
import pandas as pd

df_gouv = pd.read_csv('../data/transformed/data_gouv.csv',sep=',')
df_gouv.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
df_gouv

,DATE DE CLASSEMENT,TYPOLOGIE ÉTABLISSEMENT,CLASSEMENT,CATÉGORIE,MENTION (villages de vacances),NOM COMMERCIAL,ADRESSE,CODE POSTAL,COMMUNE,SITE INTERNET,TYPE DE SÉJOUR,CAPACITÉ D'ACCUEIL (PERSONNES),NOMBRE DE CHAMBRES,NOMBRE D'EMPLACEMENTS,NOMBRE D'UNITES D'HABITATION (résidences de tourisme),NOMBRE DE LOGEMENTS (villages de vacances),classement prorogé
0,19/06/2023,HÔTEL DE TOURISME,3 étoiles,0,0,1924 HÔTEL,2 Rue Gabriel Péri,38000,GRENOBLE,https://www.1924hotel.com/,0,62,37,0,0,0,non
1,20/10/2021,HÔTEL DE TOURISME,4 étoiles,0,0,25HOURS HÔTEL TERMINUS NORD,12 boulevard de Denain,75010,PARIS 10E ARRONDISSEMENT,0,0,489,237,0,0,0,non
2,26/09/2019,HÔTEL DE TOURISME,2 étoiles,0,0,3B HÔTEL DE BORDEAUX,66 avenue de la Libération,33440,0,0,0,26,14,0,0,0,non
3,20/03/2023,HÔTEL DE TOURISME,5 étoiles,0,0,5 TERRES HÔTEL & SPA MGALLERY BY SOFITEL,11 Place de l'Hôtel de ville,67140,BARR,0,0,54,27,0,0,0,non
4,24/10/2022,HÔTEL DE TOURISME,4 étoiles,0,0,7HÔTEL&SPA,550 Boulevard Sébastien Brandt,67400,0,https://7hotel.fr/,0,288,137,0,0,0,non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19948,12/11/2019,HÔTEL DE TOURISME,4 étoiles,0,0,ZENITUDE RELAIS & SPA LA VALADIERE,1292 ALLEE DES THERMES,34990,JUVIGNAC,0,0,272,99,0,0,0,non
19949,09/04/2021,HÔTEL DE TOURISME,3 étoiles,0,0,ZÉNITH HÔTEL CAEN,0,14000,CAEN,0,0,102,49,0,0,0,non
19950,03/10/2022,CAMPING,1 étoile,TOURISME,0,0,1 chemin du Camping,17170,0,0,0,240,0,80,0,0,non
19951,27/09/2021,CAMPING,4 étoiles,TOURISME,0,ÉCOÉTANG CAMPING DE L'ÉTANG,"603 impasse du Domaine de l'Étang, Brissac Quincé",49320,0,https://www.campingetang.com/,0,483,0,161,0,0,non


In [33]:
class AddressSearch:
    def __init__(self):
        self.SEARCH_URL = 'http://api-adresse.data.gouv.fr/search/'
        self.REVERSE_URL = 'https://api-adresse.data.gouv.fr/reverse'
        self.request_information = ''

    def search_address(self, q: str, limit: int = 5, autocomplete: int = 0,
                       search_type: str = 'municipality') -> list[dict]:
        """Search for an address in France.

        Args:
            q: Research string.
            limit: Maximum number of results.
            autocomplete: Activate/deactivate autocomplete.
            search_type: Type of results expected.

        Returns:
            Search result(s) with geocoding information.
        """
        payload = {
            'q': q,
            'limit': limit,
            'autocomplete': autocomplete,
            'type': search_type
        }

        r = requests.get(self.SEARCH_URL, params=payload)

        r.raise_for_status()

        self.request_information = f'{payload}'

        return self.format_response(r)

    def format_response(self, r: requests.Response) -> list[dict]:
        """Formats the response returned by the API.

        Args:
            r: Response object from the API.

        Returns:
            List of unpacked features dictionaries from the geojson returned by the API.
        """
        results = []
        for feature in r.json().get('features'):
            lon, lat = feature.get('geometry').get('coordinates')
            results.append({**feature.get('properties'), **{"lon": lon, "lat": lat}})
        return results


In [34]:
address_search = AddressSearch()

# Call the search_address method with the address query
results = address_search.search_address(q='1292 ALLEE DES THERMES')

# Print the results
print(results)

[{'label': 'Ax-les-Thermes', 'score': 0.4063433333333333, 'id': '09032', 'type': 'municipality', 'name': 'Ax-les-Thermes', 'postcode': '09110', 'citycode': '09032', 'x': 604384.91, 'y': 6179645.48, 'population': 1285, 'city': 'Ax-les-Thermes', 'context': '09, Ariège, Occitanie', 'importance': 0.30311, 'municipality': 'Ax-les-Thermes', 'lon': 1.834235, 'lat': 42.710052}, {'label': 'Encausse-les-Thermes', 'score': 0.36962485893416924, 'id': '31167', 'type': 'municipality', 'name': 'Encausse-les-Thermes', 'postcode': '31160', 'citycode': '31167', 'x': 516234.39, 'y': 6219847.59, 'population': 699, 'city': 'Encausse-les-Thermes', 'context': '31, Haute-Garonne, Occitanie', 'importance': 0.27277, 'municipality': 'Encausse-les-Thermes', 'lon': 0.745933, 'lat': 43.054329}, {'label': 'Thermes-Magnoac', 'score': 0.24662727272727272, 'id': '65442', 'type': 'municipality', 'name': 'Thermes-Magnoac', 'postcode': '65230', 'citycode': '65442', 'x': 505421.58, 'y': 6247200.86, 'population': 209, 'city